# Bayesian Logistic Regression
- Using Red Wine Dataset
- Uncertainty Assessment of Predictions
- https://towardsdatascience.com/uncertainty-quantification-of-predictions-with-bayesian-inference-6192e31a9fa9

In [5]:
# Import library.
import pymc3 as pm
import pandas as pd
from sklearn.datasets import load_wine

In [13]:
wine = load_wine()

df = pd.DataFrame(data=wine.data, columns=wine.feature_names)

df["target"] = wine.target

df.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0


In [15]:
# Import data.
df["target"][df["target"] > 1] = 0
df.head()

C:\Users\wallj\AppData\Local\Temp/ipykernel_7476/1361836659.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["target"][df["target"] > 1] = 0


,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0


In [18]:
df['target'].value_counts()

0    107
1     71
Name: target, dtype: int64

In [20]:
# Define model.
with pm.Model() as logistic_model:
    
    # Define priors.
    beta_0 = pm.Normal("intercept", 0, 1e6)
    beta_1 = pm.Normal("flavanoids", 0, 1e6)
    beta_2 = pm.Normal("proline", 0, 1e6)
    
    # Define posterior.
    p = pm.invlogit(beta_0 + beta_1*df["flavanoids"] +  beta_2*df["proline"])
    y = pm.Bernoulli("y", p=p, observed=df["target"])
    
    # Sample.
    trace = pm.sample(2000, tune=1000, init="adapt_diag")

C:\Users\wallj\AppData\Local\Temp/ipykernel_7476/3615933237.py:14: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  trace = pm.sample(2000, tune=1000, init="adapt_diag")
Auto-assigning NUTS sampler...
Initializing NUTS using adapt_diag...
C:\Users\wallj\anaconda3\envs\justin_env\lib\site-packages\theano\tensor\elemwise.py:826: RuntimeWarning: divide by zero encountered in log
  variables = ufunc(*ufunc_args, **ufunc_kwargs)
C:\Users\wallj\anaconda3\envs\justin_env\lib\site-packages\theano\tensor\elemwise.py:826: RuntimeWarning: invalid value encountered in multiply
  variables = ufunc(*ufunc_args, **ufunc_kwargs)
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [proline, flavanoids, intercept]


Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 6052 seconds.
